In [1]:
import os
import sys
import json
import random
import re
import string
import tqdm
import argparse
import numpy as np
import pandas as pd
from multiprocessing import Pool
#from functools import partial
from rouge_score import rouge_scorer
#from gpt3_api import make_requests as make_gpt3_requests
import transformers
from transformers import AutoTokenizer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer

import torch
from peft import PeftModel


random.seed(42)


/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#from utils.callbacks import Iteratorize, Stream
#from utils.prompter import Prompter

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "CPU"

load_8bit = True
base_model = "decapoda-research/llama-7b-hf" #"decapoda-research/llama-7b-hf"
lora_weights = "tloen/alpaca-lora-7b" #"tloen/alpaca-lora-7b"

assert (
    base_model
), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"


tokenizer = LlamaTokenizer.from_pretrained(base_model)

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        torch_dtype=torch.float16,
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        base_model, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        device_map={"": device},
    )

# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

if not load_8bit:
    model.half()  # seems to fix bugs for some users.

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:15<00:00,  2.13it/s]


In [3]:
"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output #output.split(self.template["response_split"])[1].strip()

prompt_template = "alpaca_1"
prompter = Prompter(prompt_template)

In [9]:
args = {"batch_dir": "data/gpt3_generations/",
       "seed_tasks_path": "data/galaxy-qa.jsonl",
       "num_instructions_to_generate": 1,
        "num_prompt_instructions": 5,
        "request_batch_size": 5,
        "use_clf_seed_tasks_only": False
       }
print(args)

def encode_prompt(prompt_instructions, classification=False):
    """Encode multiple prompt instructions into a single string."""
    if classification:
        prompt = "Come up with a series of classification tasks. Try to specify the possible output labels when possible.\n"
    else:
        #prompt = "Come up with a series of tasks:\n"
        prompt = "Come up with a new question using scientific keywords mentioned in existing tasks: \n"
        #"Come up with a series of new tasks using scientific keywords mentioned in existing tasks: \n" 
        #"Come up with a series of new tasks: \n"
    for idx, instruction in enumerate(prompt_instructions):
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        prompt += f"{idx+1}. {instruction}\n"
    #prompt += f"{len(prompt_instructions) + 1}."
    return prompt


def sample_machine_instructions(machine_instructions, similarities, n):
    """Sample n machine instructions from a list of machine instructions."""
    return random.sample(machine_instructions, min(n, len(machine_instructions)))


def find_word_in_string(w, s):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search(s)



{'batch_dir': 'data/gpt3_generations/', 'seed_tasks_path': 'data/galaxy-qa.jsonl', 'num_instructions_to_generate': 1, 'num_prompt_instructions': 5, 'request_batch_size': 5, 'use_clf_seed_tasks_only': False}


In [10]:
def evaluate(
    instruction,
    input=None,
    temperature=0.7, #0.1,
    top_p=0.5, #0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128, #128,
    max_tokens=1024,
    stream_output=False
):
    prompt = instruction #prompter.generate_prompt(instruction, input)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
    )

    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return prompter.get_response(output)
    #return dir(generation_output)
    #print(generation_output)
    #s = generation_output.sequences[0]
    #output = tokenizer.decode(generation_output)
    #print(s)
    #yield output #prompter.get_response(output)

In [11]:
seed_tasks = [json.loads(l) for l in open(args["seed_tasks_path"], "r")]
if args["use_clf_seed_tasks_only"]:
    seed_tasks = [t for t in seed_tasks if t["is_classification"]]
seed_instructions = [t["instruction"] for t in seed_tasks]
print(f"Loaded {len(seed_instructions)} human-written seed instructions")

os.makedirs(args["batch_dir"], exist_ok=True)

Loaded 8 human-written seed instructions


In [12]:
# similarities = {}
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)
machine_instructions = []
# now let's generate new instructions!
progress_bar = tqdm.tqdm(total=args["num_instructions_to_generate"])
if machine_instructions:
    progress_bar.update(len(machine_instructions))

with open(os.path.join(args["batch_dir"], "machine_generated_instructions.jsonl"), "a") as fout:
    #while len(machine_instructions) < args["num_instructions_to_generate"]:
    batch_inputs = []
    for _ in range(args["request_batch_size"]):
        prompt_instructions = sample_machine_instructions(
            machine_instructions, 
            similarities=None,
            n=2)
        # sample human instructions from the pool
        prompt_instructions += random.sample(seed_instructions, args["num_prompt_instructions"] - len(prompt_instructions))
        random.shuffle(prompt_instructions)
        prompt = encode_prompt(prompt_instructions, classification=args["use_clf_seed_tasks_only"])
        print("Response:", evaluate(prompt))
        print("---------")


  0%|                                                                                                                                             | 0/1 [15:36<?, ?it/s]


Come up with a new question using scientific keywords mentioned in existing tasks: 
1. Are UMIs not specific to certain genes? Can the same UMI map to different genes?
2. Imagine that we forgot to filter the VCFs to contain only fixed variants, and there are also SNPs with frequencies, of 15%, 30%, or 56.78%. Which allele do you think bcftools consensus would insert in the genome?
3. What do UMIs do?
4. TB Variant Filter reads the VCF and output only SNPs that have, at least, 90% frequency. How can the software extract such information from the VCF datasets?
5. Why do we need to barcode a read transcript too? Isn’t mapping it against the reference genome enough?

Response: <unk>Come up with a new question using scientific keywords mentioned in existing tasks: 
1. Are UMIs not specific to certain genes? Can the same UMI map to different genes?
2. Imagine that we forgot to filter the VCFs to contain only fixed variants, and there are also SNPs with frequencies, of 15%, 30%, or 56.78%. Wh

In [ ]:
# testing code for readme
#for instruction in [
#    "Tell me about alpacas.",
#    "Tell me about the president of Mexico in 2019.",
    #"Tell me about the king of France in 2019.",
    #"List all Canadian provinces in alphabetical order.",
    #"Write a Python program that prints the first 10 Fibonacci numbers.",
    #"Write a program that prints the numbers from 1 to 100. But for multiples of three print 'Fizz' instead of the number and for the multiples of five print 'Buzz'. For numbers which are multiples of both three and five print 'FizzBuzz'.",  # noqa: E501
    #"Tell me five words that rhyme with 'shock'.",
    #"Translate the sentence 'I have no mouth but I must scream' into Spanish.",
    #"Count up from 1 to 500.",
#]:
#    print("Instruction:", instruction)
#    print("Response:", evaluate(instruction))
#    print()

In [ ]:
'''seed_tasks = [json.loads(l) for l in open(args["seed_tasks_path"], "r")]
if args["use_clf_seed_tasks_only"]:
    seed_tasks = [t for t in seed_tasks if t["is_classification"]]
seed_instructions = [t["instruction"] for t in seed_tasks]
print(f"Loaded {len(seed_instructions)} human-written seed instructions")

os.makedirs(args["batch_dir"], exist_ok=True)
request_idx = 0
# load the LM-generated instructions
machine_instructions = []
if os.path.exists(os.path.join(args["batch_dir"], "machine_generated_instructions.jsonl")):
    with open(os.path.join(args["batch_dir"], "machine_generated_instructions.jsonl"), "r") as fin:
        for line in fin:
            instruction_info = json.loads(line)
            machine_instructions.append(instruction_info["instruction"])
            request_idx = instruction_info["request_idx"] + 1
    print(f"Loaded {len(machine_instructions)} machine-generated instructions")

# similarities = {}
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

# now let's generate new instructions!
progress_bar = tqdm.tqdm(total=args["num_instructions_to_generate"])
if machine_instructions:
    progress_bar.update(len(machine_instructions))

with open(os.path.join(args["batch_dir"], "machine_generated_instructions.jsonl"), "a") as fout:
    #while len(machine_instructions) < args["num_instructions_to_generate"]:
    batch_inputs = []
    for _ in range(args["request_batch_size"]):
        # sample machine instructions from the pool
        prompt_instructions = sample_machine_instructions(
            machine_instructions, 
            similarities=None,
            n=2)
        print("prompt_instructions")
        print(prompt_instructions)
        print()
        # sample human instructions from the pool
        prompt_instructions += random.sample(seed_instructions, args["num_prompt_instructions"] - len(prompt_instructions))
        random.shuffle(prompt_instructions)
        prompt = encode_prompt(prompt_instructions, classification=args["use_clf_seed_tasks_only"])
        print("prompt")
        print(prompt)
        print()
        batch_inputs.append(prompt)

        sequences = pipeline(
            #'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
            batch_inputs,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=200,
        )
        print("Responses:")
        print(sequences)
        break'''